In [9]:
pwd

u'/Users/naveenkumar/Dropbox/OnlineWork/mlusecase/Notebooks/Simility'

In [1]:
%matplotlib inline
import pandas as pd
import os
import dateutil
import warnings
import csv
import numpy as np
import re
import gc

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding
import hypertools as hyp

from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

import datetime as dt
import pandas as pd
import numpy as np
import warnings
import string

import 

warnings.filterwarnings('ignore')

wdir = "/Users/naveenkumar/simility/quoka"
INSTANCES = os.path.join(wdir, "quoka_uniq_filtered_cols.csv")



Using TensorFlow backend.


In [54]:
# Calculate information value
def calc_iv(df, feature, target, pr=0):

    lst = []

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature, val, df[df[feature] == val].count()[feature], df[(df[feature] == val) & (df[target] == 1)].count()[feature]])

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Bad'])
    data = data[data['Bad'] > 0]

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
    data['ValIV'] = (data['WoE'] * (data['Distribution Good'] - data['Distribution Bad']))

    data = data.sort_values(by=['Variable', 'Value'], ascending=True)

    IV = data['ValIV'].sum()
    
    if pr == 1:
        print(data)
    
    return IV

In [2]:
d = pd.read_csv(INSTANCES,sep=',',header='infer',quotechar='"', low_memory=False)

In [3]:
print(d['sim_dc'].value_counts())
print(d['eid'].nunique())#4506988
print(d.shape[0])#4506988

{'OK'}                     343230
{'SCORING_DELETE'}          13932
{'OK','SCORING_DELETE'}       942
{}                            213
Name: sim_dc, dtype: int64
358317
358317


In [24]:
d['y'] = np.where((d['sim_dc'] == "{'SCORING_DELETE'}") ,1,0)
d['y'] = np.where((d['sim_dc'] == "{'OK','SCORING_DELETE'}"),1,d['y'])

In [131]:
#pd.crosstab(d['sim_dc'], d['y'])

In [26]:
d['y'].value_counts()

0    343443
1     14874
Name: y, dtype: int64

In [28]:
d = d.drop('sim_dc', axis = 1)

In [33]:
percentiles_list = [0.01, 0.02, 0.05, 0.25, 0.50, 0.75, 0.95, 0.98, 0.99]
x_d_allsummary = d.describe(percentiles=percentiles_list, include ='all').transpose()
x_d_allsummary.reset_index(level=0, inplace=True)
x_d_allsummary.rename(columns={'index': 'ColName'}, inplace=True)
x_d_allsummary['DataType'] = x_d_allsummary['ColName'].apply(lambda col:d[col].dtype)
x_d_allsummary['TotalNullValues'] = x_d_allsummary['ColName'].apply(lambda col:d[col].isnull().sum())

In [34]:
x_d_allsummary['DataType'].value_counts()

float64    1442
int64        21
bool          3
object        1
Name: DataType, dtype: int64

In [127]:
d.dtypes[d.dtypes == "object"]

ad_zipcode    object
dtype: object

In [61]:
d = d.fillna(0)

In [387]:
percentiles_list = [0.01, 0.02, 0.05, 0.25, 0.50, 0.75, 0.95, 0.98, 0.99]
d_summ = d.describe(percentiles=percentiles_list, include ='all').transpose()
d_summ.reset_index(level=0, inplace=True)
d_summ.rename(columns={'index': 'ColName'}, inplace=True)
d_summ['DataType'] = d_summ['ColName'].apply(lambda col:d[col].dtype)
d_summ['TotalNullValues'] = d_summ['ColName'].apply(lambda col:d[col].isnull().sum())
d_summ['TotalUniqueLevels'] = d_summ['ColName'].apply(lambda col:d[col].nunique())
d_summ['PercentUniqueValues'] = d_summ['TotalUniqueLevels']/d_summ.shape[0]

In [412]:
sim_ll_cols = list(filter(re.compile(".*(sim_ll)").match , mylist))

def compute_iv_ds(df_in, target_col, ignore_cols=[], n_quantile=5):
    
    y_d = d[target_col].to_frame()
    
    iv_df = pd.DataFrame()
    
    f_list = df_in.columns.tolist()
    f_list.remove(target_col)
    
    f_list = list(set(f_list) - set(ignore_cols))
    
    
    for f in f_list:
        f_dtype = df_in[f].dtype.name
        
        if df_in[f].max() > df_in[f].min():
            if re.sub(r'(float|int)[0-9]+', r'float', f_dtype) == 'float':
        
                tmp_df = pd.qcut(d[f], n_quantile, labels=None, retbins=False, precision=3, duplicates='drop').to_frame()
                tmp_df = tmp_df.join(y_d)

                iv_out = np.round(calc_iv(tmp_df, f, target_col),2)
                
                tmp_iv_df = pd.DataFrame({'feature' : [f],
                                          'datatype' :[d[f].dtype],
                                           'iv' : [iv_out]
                                         })
                iv_df = iv_df.append(tmp_iv_df)

    
            elif re.sub(r'.*bool.*', r'bool', f_dtype) == 'bool':

                    tmp_df = d[f].to_frame()
                    tmp_df = tmp_df.join(y_d)
                    iv_out = np.round(calc_iv(tmp_df, f, target_col),2)

                    tmp_iv_df = pd.DataFrame({'feature'  : [f],
                                              'datatype' :[d[f].dtype],
                                               'iv'      : [iv_out]
                                             })

                    iv_df = iv_df.append(tmp_iv_df)

            else :
                print('IV computattion of '+ f_dtype + ' is under progess')
            
    iv_df.reset_index()

    return iv_df 


d_sample_iv = compute_iv_ds(d, 'y', sim_ll_cols, 5)

#d_iv = compute_iv_ds(d, 'y',5)

IV computattion of object is under progess


In [413]:
print(d_iv.shape)
print(d_sample_iv.shape)

(1435, 4)
(1291, 3)


In [404]:
# Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
import pandas
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# load data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
# feature extraction

SelectKBest(score_func=chi2, k=4)

SelectKBest(k=4, score_func=<function chi2 at 0x1a155928c0>)

In [ ]:
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)
# summarize scores
numpy.set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:5,:])

In [407]:
mylist = d.columns.tolist()
type(mylist)


list

['sim_ll_LongtermCustomer',
 'sim_ll_SuspiciousAdText',
 'sim_ll_HighEmailKbDistance',
 'sim_ll_SuspiciousAdHeadline',
 'sim_ll_HighAdSessionPerUserName',
 'sim_ll_SuspiciousUserEmailDomain',
 'sim_ll_HighAdSessionPerAdDeviceid',
 'sim_ll_HighUserPhonePerAdDeviceid',
 'sim_ll_DeviceIdCityPostingCityMismatch',
 'sim_ll_SuspiciousUserAddress',
 'sim_ll_DHL_ip_range',
 'sim_ll_BadGMXDomains',
 'sim_ll_AdTextContainsPhone',
 'sim_ll_HighUserPhonePerAdSession',
 'sim_ll_HighUserPhonePerAdIpaddress',
 'sim_ll_HighAdSessionPerAdEmail',
 'sim_ll_HighAdSessionPerAdHeadline',
 'sim_ll_CustomerRegistrationBeforeAdTaking_24h',
 'sim_ll_HighAdImagehashPerAdDeviceid',
 'sim_ll_HighAdImagehashPerAdSession',
 'sim_ll_HighAdImagehashPerAdIpaddress',
 'sim_ll_SuspiciousUserName',
 'sim_ll_SuspiciousAdAddress',
 'sim_ll_HighAdSessionPerAdIpaddress',
 'sim_ll_HighAverageItemValuePerIp1Day',
 'sim_ll_HighAdDeviceidPerAdIpaddress',
 'sim_ll_HighAdImagehashPerAdText',
 'sim_ll_HighAdImagehashPerAdHeadline',
